# Notebook 76: 勾配の病理学 ― 消失・爆発・鞍点

## Gradient Pathology: Vanishing, Exploding, and Saddle Points

---

### このノートブックの位置づけ

**Unit 0.0「ニューラルエンジンの深部」** の最終章として、学習が失敗するメカニズムを理解し、診断・対策の方法を学びます。

### 学習目標

1. **勾配消失問題** のメカニズムと対策を理解する
2. **勾配爆発問題** のメカニズムと対策を理解する
3. **鞍点と局所解** の概念を理解する
4. **勾配のモニタリング** ツールを実装する

### 前提知識

- Notebook 70-75 の内容

---

## 目次

1. [勾配消失問題](#1-勾配消失問題)
2. [勾配爆発問題](#2-勾配爆発問題)
3. [活性化関数の選択](#3-活性化関数の選択)
4. [重みの初期化](#4-重みの初期化)
5. [鞍点と局所解](#5-鞍点と局所解)
6. [勾配モニタリングツール](#6-勾配モニタリングツール)
7. [最適化手法の比較](#7-最適化手法の比較)
8. [演習問題](#8-演習問題)
9. [Unit 0.0 総まとめ](#9-unit-00-総まとめ)

In [ ]:
# 環境セットアップ
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from abc import ABC, abstractmethod
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

np.random.seed(42)

print("環境セットアップ完了")

In [ ]:
# 前のノートブックのクラスを再定義

class Layer(ABC):
    def __init__(self):
        self.params = {}
        self.grads = {}
        self.cache = {}
    
    @abstractmethod
    def forward(self, x): pass
    
    @abstractmethod
    def backward(self, dout): pass


class Linear(Layer):
    def __init__(self, in_features, out_features, init='xavier'):
        super().__init__()
        if init == 'xavier':
            scale = np.sqrt(2.0 / (in_features + out_features))
        elif init == 'he':
            scale = np.sqrt(2.0 / in_features)
        elif init == 'normal':
            scale = 0.01
        else:
            scale = 1.0
        
        self.params['W'] = np.random.randn(in_features, out_features) * scale
        self.params['b'] = np.zeros(out_features)
        self.grads['W'] = None
        self.grads['b'] = None
    
    def forward(self, x):
        self.cache['x'] = x
        return x @ self.params['W'] + self.params['b']
    
    def backward(self, dout):
        x = self.cache['x']
        self.grads['W'] = x.T @ dout
        self.grads['b'] = np.sum(dout, axis=0)
        return dout @ self.params['W'].T


class Sigmoid(Layer):
    def forward(self, x):
        y = 1 / (1 + np.exp(-np.clip(x, -500, 500)))
        self.cache['y'] = y
        return y
    
    def backward(self, dout):
        y = self.cache['y']
        return dout * y * (1 - y)


class ReLU(Layer):
    def forward(self, x):
        self.cache['mask'] = (x > 0).astype(float)
        return np.maximum(0, x)
    
    def backward(self, dout):
        return dout * self.cache['mask']


class Tanh(Layer):
    def forward(self, x):
        y = np.tanh(x)
        self.cache['y'] = y
        return y
    
    def backward(self, dout):
        y = self.cache['y']
        return dout * (1 - y ** 2)


class MSELoss:
    def __init__(self):
        self.cache = {}
    
    def forward(self, y, t):
        self.cache['y'] = y
        self.cache['t'] = t
        self.cache['N'] = y.shape[0]
        return np.mean((y - t) ** 2)
    
    def backward(self):
        y, t, N = self.cache['y'], self.cache['t'], self.cache['N']
        return (2 / N) * (y - t)


print("Layer クラス群を定義しました")

---

## 1. 勾配消失問題

### 1.1 問題の概要

深いネットワークでは、逆伝播中に勾配が層を通過するたびに **小さくなっていく** 問題が発生します。

**原因**：シグモイドやtanhの導関数は最大でも1未満
- $\sigma'(x) \leq 0.25$（シグモイド）
- $\tanh'(x) \leq 1$（tanh）

n層のネットワークでは、勾配が $0.25^n$ 倍に減衰する可能性があります。

In [ ]:
# シグモイドの導関数の最大値を確認
x = np.linspace(-6, 6, 1000)
sigmoid = 1 / (1 + np.exp(-x))
sigmoid_deriv = sigmoid * (1 - sigmoid)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# シグモイドと導関数
axes[0].plot(x, sigmoid, label=r'$\sigma(x)$', linewidth=2)
axes[0].plot(x, sigmoid_deriv, label=r"$\sigma'(x)$", linewidth=2)
axes[0].axhline(y=0.25, color='red', linestyle='--', alpha=0.7, label='最大値 0.25')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('シグモイド関数とその導関数')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 勾配の減衰
n_layers = np.arange(1, 21)
gradient_decay_025 = 0.25 ** n_layers  # 最悪ケース
gradient_decay_05 = 0.5 ** n_layers    # 中間ケース

axes[1].semilogy(n_layers, gradient_decay_025, 'o-', label=r'$0.25^n$ (最悪)', linewidth=2)
axes[1].semilogy(n_layers, gradient_decay_05, 's-', label=r'$0.5^n$', linewidth=2)
axes[1].set_xlabel('層の数')
axes[1].set_ylabel('勾配の大きさ（対数スケール）')
axes[1].set_title('勾配消失: 層が深くなるほど勾配が減衰')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"10層の場合: 勾配は最大 {0.25**10:.2e} 倍に減衰")
print(f"20層の場合: 勾配は最大 {0.25**20:.2e} 倍に減衰")

### 1.2 深いネットワークでの実験

In [ ]:
def build_deep_network(n_layers, activation='sigmoid', init='normal'):
    """深いネットワークを構築"""
    layers = []
    act_class = {'sigmoid': Sigmoid, 'relu': ReLU, 'tanh': Tanh}[activation]
    
    # 入力層
    layers.append(Linear(2, 10, init=init))
    layers.append(act_class())
    
    # 隠れ層
    for _ in range(n_layers - 2):
        layers.append(Linear(10, 10, init=init))
        layers.append(act_class())
    
    # 出力層
    layers.append(Linear(10, 1, init=init))
    
    return layers


def forward_backward(layers, X, t):
    """順伝播・逆伝播を実行し、各層の勾配ノルムを記録"""
    loss_fn = MSELoss()
    
    # 順伝播
    h = X
    for layer in layers:
        h = layer.forward(h)
    
    loss = loss_fn.forward(h, t)
    
    # 逆伝播
    dout = loss_fn.backward()
    gradient_norms = []
    
    for layer in reversed(layers):
        dout = layer.backward(dout)
        if hasattr(layer, 'grads') and layer.grads.get('W') is not None:
            grad_norm = np.linalg.norm(layer.grads['W'])
            gradient_norms.append(grad_norm)
    
    return loss, gradient_norms[::-1]  # 入力側から順に


# 勾配消失の実験
np.random.seed(42)
X = np.random.randn(32, 2)
t = np.random.randn(32, 1)

# シグモイドで10層
layers_sigmoid = build_deep_network(10, activation='sigmoid', init='normal')
_, grad_norms_sigmoid = forward_backward(layers_sigmoid, X, t)

# ReLUで10層
np.random.seed(42)
layers_relu = build_deep_network(10, activation='relu', init='he')
_, grad_norms_relu = forward_backward(layers_relu, X, t)

# 可視化
fig, ax = plt.subplots(figsize=(10, 6))
layer_indices = np.arange(1, len(grad_norms_sigmoid) + 1)

ax.bar(layer_indices - 0.2, grad_norms_sigmoid, width=0.4, label='Sigmoid', alpha=0.8)
ax.bar(layer_indices + 0.2, grad_norms_relu, width=0.4, label='ReLU', alpha=0.8)

ax.set_xlabel('層番号（入力側から）')
ax.set_ylabel('勾配のノルム')
ax.set_title('各層の勾配ノルム: Sigmoid vs ReLU')
ax.set_yscale('log')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("【観察】")
print(f"Sigmoid: 入力側の勾配 = {grad_norms_sigmoid[0]:.2e}（消失）")
print(f"ReLU:    入力側の勾配 = {grad_norms_relu[0]:.2e}（維持）")

---

## 2. 勾配爆発問題

### 2.1 問題の概要

勾配消失とは逆に、勾配が **大きくなりすぎる** 問題もあります。

**原因**：
- 重みの初期値が大きすぎる
- 特定のネットワーク構造（RNNなど）

**症状**：
- 損失が NaN になる
- パラメータが発散する

In [ ]:
# 勾配爆発の実験
np.random.seed(42)

# 大きな初期値
layers_explode = build_deep_network(10, activation='relu', init='large')
# 手動で大きな重みを設定
for layer in layers_explode:
    if hasattr(layer, 'params'):
        layer.params['W'] = np.random.randn(*layer.params['W'].shape) * 2.0

loss, grad_norms_explode = forward_backward(layers_explode, X, t)

# 適切な初期値
np.random.seed(42)
layers_normal = build_deep_network(10, activation='relu', init='he')
_, grad_norms_normal = forward_backward(layers_normal, X, t)

# 可視化
fig, ax = plt.subplots(figsize=(10, 6))

ax.bar(layer_indices - 0.2, grad_norms_explode, width=0.4, label='大きい初期値', alpha=0.8, color='red')
ax.bar(layer_indices + 0.2, grad_norms_normal, width=0.4, label='He初期化', alpha=0.8, color='green')

ax.set_xlabel('層番号（入力側から）')
ax.set_ylabel('勾配のノルム（対数スケール）')
ax.set_title('勾配爆発: 大きな初期値の危険性')
ax.set_yscale('log')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"大きい初期値: 最大勾配ノルム = {max(grad_norms_explode):.2e}")
print(f"He初期化:     最大勾配ノルム = {max(grad_norms_normal):.2e}")

### 2.2 勾配クリッピング

In [ ]:
def clip_gradients(layers, max_norm=1.0):
    """
    勾配クリッピング: 勾配のノルムが閾値を超えたらスケーリング
    
    Args:
        layers: レイヤーのリスト
        max_norm: 最大ノルム
    """
    # 全勾配のノルムを計算
    total_norm = 0
    for layer in layers:
        if hasattr(layer, 'grads'):
            for name, grad in layer.grads.items():
                if grad is not None:
                    total_norm += np.sum(grad ** 2)
    total_norm = np.sqrt(total_norm)
    
    # クリッピング
    clip_coef = max_norm / (total_norm + 1e-6)
    if clip_coef < 1:
        for layer in layers:
            if hasattr(layer, 'grads'):
                for name in layer.grads:
                    if layer.grads[name] is not None:
                        layer.grads[name] *= clip_coef
    
    return total_norm, min(clip_coef, 1.0)


# クリッピングの効果を確認
np.random.seed(42)
layers_test = build_deep_network(10, activation='relu', init='large')
for layer in layers_test:
    if hasattr(layer, 'params'):
        layer.params['W'] = np.random.randn(*layer.params['W'].shape) * 2.0

_, _ = forward_backward(layers_test, X, t)

# クリッピング前
grad_norms_before = []
for layer in layers_test:
    if hasattr(layer, 'grads') and layer.grads.get('W') is not None:
        grad_norms_before.append(np.linalg.norm(layer.grads['W']))

# クリッピング実行
total_norm, clip_coef = clip_gradients(layers_test, max_norm=1.0)

# クリッピング後
grad_norms_after = []
for layer in layers_test:
    if hasattr(layer, 'grads') and layer.grads.get('W') is not None:
        grad_norms_after.append(np.linalg.norm(layer.grads['W']))

print(f"【勾配クリッピング】")
print(f"クリッピング前の総ノルム: {total_norm:.2f}")
print(f"クリッピング係数: {clip_coef:.4f}")
print(f"クリッピング後の最大勾配: {max(grad_norms_after):.4f}")

---

## 3. 活性化関数の選択

### 3.1 各活性化関数の特性比較

In [ ]:
# 活性化関数の比較
x = np.linspace(-4, 4, 1000)

activations = {
    'Sigmoid': (1 / (1 + np.exp(-x)), lambda y: y * (1 - y)),
    'Tanh': (np.tanh(x), lambda y: 1 - y**2),
    'ReLU': (np.maximum(0, x), lambda _: (x > 0).astype(float)),
    'Leaky ReLU': (np.where(x > 0, x, 0.01 * x), lambda _: np.where(x > 0, 1, 0.01)),
}

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for ax, (name, (y, deriv_fn)) in zip(axes.flat, activations.items()):
    deriv = deriv_fn(y)
    
    ax.plot(x, y, label='f(x)', linewidth=2)
    ax.plot(x, deriv, label="f'(x)", linewidth=2, linestyle='--')
    ax.axhline(y=0, color='black', linewidth=0.5)
    ax.axvline(x=0, color='black', linewidth=0.5)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(name)
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_xlim(-4, 4)
    ax.set_ylim(-2, 2)

plt.tight_layout()
plt.show()

print("【活性化関数の特性】")
print("")
print("Sigmoid:")
print("  - 出力範囲: (0, 1)")
print("  - 問題: 勾配消失、計算コスト")
print("")
print("Tanh:")
print("  - 出力範囲: (-1, 1)")
print("  - Sigmoidより勾配消失しにくい")
print("")
print("ReLU:")
print("  - 出力範囲: [0, ∞)")
print("  - 計算が速い、勾配消失しにくい")
print("  - 問題: 死んだニューロン (x < 0 で勾配が0)")
print("")
print("Leaky ReLU:")
print("  - x < 0 でも小さな勾配を保持")
print("  - 死んだニューロン問題を軽減")

---

## 4. 重みの初期化

### 4.1 初期化手法の比較

In [ ]:
def analyze_activations(layers, X, title):
    """各層の活性化の分布を分析"""
    activations = []
    h = X
    
    for layer in layers:
        h = layer.forward(h)
        if isinstance(layer, (Sigmoid, ReLU, Tanh)):
            activations.append(h.copy())
    
    return activations


# 異なる初期化での活性化分布
np.random.seed(42)
X_test = np.random.randn(1000, 2)

init_methods = {
    '標準正規 (σ=1)': 'large',
    '小さい (σ=0.01)': 'normal',
    'Xavier': 'xavier',
    'He': 'he',
}

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for ax, (name, init) in zip(axes.flat, init_methods.items()):
    np.random.seed(42)
    layers = build_deep_network(5, activation='sigmoid', init=init if init != 'large' else 'normal')
    
    # 大きな初期値の場合は手動設定
    if init == 'large':
        for layer in layers:
            if hasattr(layer, 'params'):
                layer.params['W'] = np.random.randn(*layer.params['W'].shape) * 1.0
    
    activations = analyze_activations(layers, X_test, name)
    
    for i, act in enumerate(activations):
        ax.hist(act.flatten(), bins=50, alpha=0.5, label=f'層 {i+1}')
    
    ax.set_xlabel('活性化値')
    ax.set_ylabel('頻度')
    ax.set_title(f'{name}\n(std={np.std(activations[-1]):.4f})')
    ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

print("【初期化の指針】")
print("- Sigmoid/Tanh: Xavier初期化 (σ = √(2/(n_in + n_out)))")
print("- ReLU: He初期化 (σ = √(2/n_in))")

---

## 5. 鞍点と局所解

### 5.1 損失曲面の可視化

In [ ]:
# 鞍点を持つ関数の可視化
def saddle_function(x, y):
    """鞍点を持つ関数: f(x,y) = x² - y²"""
    return x**2 - y**2


# メッシュグリッド
x = np.linspace(-2, 2, 100)
y = np.linspace(-2, 2, 100)
X_mesh, Y_mesh = np.meshgrid(x, y)
Z = saddle_function(X_mesh, Y_mesh)

fig = plt.figure(figsize=(14, 5))

# 3Dプロット
ax1 = fig.add_subplot(1, 2, 1, projection='3d')
ax1.plot_surface(X_mesh, Y_mesh, Z, cmap='coolwarm', alpha=0.8)
ax1.scatter([0], [0], [0], color='black', s=100, label='鞍点')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('f(x,y)')
ax1.set_title(r'鞍点: $f(x,y) = x^2 - y^2$')
ax1.view_init(elev=25, azim=45)

# 等高線
ax2 = fig.add_subplot(1, 2, 2)
contour = ax2.contour(X_mesh, Y_mesh, Z, levels=20, cmap='coolwarm')
ax2.scatter([0], [0], color='black', s=100, zorder=5, label='鞍点')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_title('等高線図')
ax2.set_aspect('equal')
ax2.legend()
plt.colorbar(contour, ax=ax2)

plt.tight_layout()
plt.show()

print("【鞍点の特徴】")
print("- 勾配がゼロになる点")
print("- ある方向では極小、別の方向では極大")
print("- 高次元空間では局所解より鞍点の方が多い")

### 5.2 最適化の軌跡

In [ ]:
# 複雑な損失曲面での最適化
def complex_loss(x, y):
    """複数の極小を持つ関数"""
    return (x**2 + y - 11)**2 + (x + y**2 - 7)**2  # Himmelblau関数


def gradient_complex(x, y):
    dx = 2 * (x**2 + y - 11) * 2 * x + 2 * (x + y**2 - 7)
    dy = 2 * (x**2 + y - 11) + 2 * (x + y**2 - 7) * 2 * y
    return np.array([dx, dy])


def optimize_trajectory(start, lr=0.01, n_steps=100):
    """勾配降下の軌跡を記録"""
    path = [np.array(start)]
    pos = np.array(start, dtype=float)
    
    for _ in range(n_steps):
        grad = gradient_complex(pos[0], pos[1])
        pos = pos - lr * grad
        path.append(pos.copy())
    
    return np.array(path)


# 等高線
x = np.linspace(-5, 5, 200)
y = np.linspace(-5, 5, 200)
X_mesh, Y_mesh = np.meshgrid(x, y)
Z = complex_loss(X_mesh, Y_mesh)

fig, ax = plt.subplots(figsize=(10, 8))

contour = ax.contour(X_mesh, Y_mesh, np.log(Z + 1), levels=30, cmap='viridis', alpha=0.7)

# 異なる開始点からの軌跡
start_points = [(-4, -4), (4, 4), (-4, 4), (4, -4), (0, 0)]
colors = ['red', 'blue', 'green', 'orange', 'purple']

for start, color in zip(start_points, colors):
    path = optimize_trajectory(start, lr=0.01, n_steps=200)
    ax.plot(path[:, 0], path[:, 1], '-', color=color, linewidth=2, alpha=0.8)
    ax.scatter(path[0, 0], path[0, 1], color=color, s=100, marker='o', edgecolors='black')
    ax.scatter(path[-1, 0], path[-1, 1], color=color, s=100, marker='*', edgecolors='black')

# 真の極小点をマーク
minima = [(3, 2), (-2.805, 3.131), (-3.779, -3.283), (3.584, -1.848)]
for m in minima:
    ax.scatter(*m, color='black', s=200, marker='X', zorder=10)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Himmelblau関数: 複数の極小点への収束\n(○: 開始点, ★: 終了点, X: 真の極小点)')
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)

plt.tight_layout()
plt.show()

print("【観察】")
print("- 開始点によって収束先が異なる")
print("- 必ずしもグローバル最小点に到達しない")

---

## 6. 勾配モニタリングツール

### 6.1 GradientMonitorクラス

In [ ]:
class GradientMonitor:
    """
    学習中の勾配をモニタリングするツール
    
    機能:
    - 各層の勾配ノルムの記録
    - 勾配消失・爆発の検出
    - 可視化
    """
    
    def __init__(self):
        self.history = []  # [(epoch, {layer_name: grad_norm}), ...]
    
    def record(self, epoch, layers):
        """各層の勾配ノルムを記録"""
        grad_norms = {}
        layer_idx = 0
        
        for layer in layers:
            if hasattr(layer, 'grads') and layer.grads.get('W') is not None:
                name = f'Linear_{layer_idx}'
                grad_norms[name] = np.linalg.norm(layer.grads['W'])
                layer_idx += 1
        
        self.history.append((epoch, grad_norms))
    
    def detect_problems(self, vanish_threshold=1e-7, explode_threshold=1e3):
        """勾配の問題を検出"""
        if not self.history:
            return []
        
        problems = []
        epoch, grad_norms = self.history[-1]
        
        for name, norm in grad_norms.items():
            if norm < vanish_threshold:
                problems.append(f"警告: {name} で勾配消失の可能性 (norm={norm:.2e})")
            elif norm > explode_threshold:
                problems.append(f"警告: {name} で勾配爆発の可能性 (norm={norm:.2e})")
        
        return problems
    
    def plot(self):
        """勾配ノルムの推移を可視化"""
        if not self.history:
            print("記録がありません")
            return
        
        epochs = [h[0] for h in self.history]
        layer_names = list(self.history[0][1].keys())
        
        fig, ax = plt.subplots(figsize=(12, 6))
        
        for name in layer_names:
            norms = [h[1][name] for h in self.history]
            ax.plot(epochs, norms, label=name, linewidth=2)
        
        ax.set_xlabel('Epoch')
        ax.set_ylabel('勾配ノルム')
        ax.set_title('各層の勾配ノルムの推移')
        ax.set_yscale('log')
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        return fig, ax


print("GradientMonitor クラスを定義しました")

In [ ]:
# GradientMonitorの使用例
np.random.seed(42)

# データ
X = np.random.randn(100, 2)
t = np.random.randn(100, 1)

# モデル
layers = build_deep_network(8, activation='sigmoid', init='xavier')
loss_fn = MSELoss()
monitor = GradientMonitor()

# 学習ループ（簡易版）
lr = 0.1
for epoch in range(100):
    # 順伝播
    h = X
    for layer in layers:
        h = layer.forward(h)
    loss = loss_fn.forward(h, t)
    
    # 逆伝播
    dout = loss_fn.backward()
    for layer in reversed(layers):
        dout = layer.backward(dout)
    
    # 勾配をモニタリング
    monitor.record(epoch, layers)
    
    # パラメータ更新
    for layer in layers:
        if hasattr(layer, 'params'):
            for name in layer.params:
                if layer.grads.get(name) is not None:
                    layer.params[name] -= lr * layer.grads[name]

# 可視化
monitor.plot()
plt.show()

# 問題の検出
problems = monitor.detect_problems()
if problems:
    print("\n検出された問題:")
    for p in problems:
        print(f"  {p}")
else:
    print("\n問題は検出されませんでした")

---

## 7. 最適化手法の比較

### 7.1 鞍点での挙動比較

In [ ]:
# SGD vs Momentum vs Adam の鞍点での挙動

def sgd_step(pos, grad, lr=0.1, state=None):
    return pos - lr * grad, state


def momentum_step(pos, grad, lr=0.1, state=None, momentum=0.9):
    if state is None:
        state = {'v': np.zeros_like(pos)}
    state['v'] = momentum * state['v'] - lr * grad
    return pos + state['v'], state


def adam_step(pos, grad, lr=0.01, state=None, beta1=0.9, beta2=0.999, eps=1e-8):
    if state is None:
        state = {'m': np.zeros_like(pos), 'v': np.zeros_like(pos), 't': 0}
    state['t'] += 1
    state['m'] = beta1 * state['m'] + (1 - beta1) * grad
    state['v'] = beta2 * state['v'] + (1 - beta2) * (grad ** 2)
    m_hat = state['m'] / (1 - beta1 ** state['t'])
    v_hat = state['v'] / (1 - beta2 ** state['t'])
    return pos - lr * m_hat / (np.sqrt(v_hat) + eps), state


def run_optimizer(step_fn, start, loss_fn, grad_fn, n_steps=100, **kwargs):
    path = [np.array(start)]
    pos = np.array(start, dtype=float)
    state = None
    
    for _ in range(n_steps):
        grad = grad_fn(pos[0], pos[1])
        pos, state = step_fn(pos, grad, state=state, **kwargs)
        path.append(pos.copy())
    
    return np.array(path)


# 鞍点関数
def saddle_grad(x, y):
    return np.array([2*x, -2*y])


# 各最適化手法の軌跡
start = (0.1, 0.1)

path_sgd = run_optimizer(sgd_step, start, saddle_function, saddle_grad, n_steps=50, lr=0.1)
path_momentum = run_optimizer(momentum_step, start, saddle_function, saddle_grad, n_steps=50, lr=0.1)
path_adam = run_optimizer(adam_step, start, saddle_function, saddle_grad, n_steps=50, lr=0.5)

# 可視化
x = np.linspace(-2, 2, 100)
y = np.linspace(-2, 2, 100)
X_mesh, Y_mesh = np.meshgrid(x, y)
Z = saddle_function(X_mesh, Y_mesh)

fig, ax = plt.subplots(figsize=(10, 8))

contour = ax.contour(X_mesh, Y_mesh, Z, levels=20, cmap='coolwarm', alpha=0.7)

ax.plot(path_sgd[:, 0], path_sgd[:, 1], 'o-', label='SGD', linewidth=2, markersize=4)
ax.plot(path_momentum[:, 0], path_momentum[:, 1], 's-', label='Momentum', linewidth=2, markersize=4)
ax.plot(path_adam[:, 0], path_adam[:, 1], '^-', label='Adam', linewidth=2, markersize=4)

ax.scatter([0], [0], color='black', s=200, marker='X', zorder=10, label='鞍点')

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('鞍点周辺での最適化手法の挙動')
ax.legend()
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)

plt.tight_layout()
plt.show()

print("【観察】")
print("- SGD: 鞍点に吸い込まれやすい")
print("- Momentum: 慣性で鞍点を通過しやすい")
print("- Adam: 適応的な学習率で効率的に脱出")

---

## 8. 演習問題

### 演習 8.1: Batch Normalizationの実装

勾配消失を軽減する Batch Normalization を実装してください。

In [ ]:
# 演習 8.1: 解答欄

class BatchNorm(Layer):
    """
    Batch Normalization
    
    y = γ * (x - μ) / √(σ² + ε) + β
    
    TODO: forward() と backward() を実装
    """
    
    def __init__(self, n_features, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.params['gamma'] = np.ones(n_features)
        self.params['beta'] = np.zeros(n_features)
        self.grads['gamma'] = None
        self.grads['beta'] = None
    
    def forward(self, x):
        # TODO: 実装
        pass
    
    def backward(self, dout):
        # TODO: 実装
        pass

### 演習 8.2: 異なる深さでの学習比較

5層、10層、20層のネットワークで学習を比較し、勾配消失の影響を観察してください。

In [ ]:
# 演習 8.2: 解答欄

# TODO: 異なる深さのネットワークを構築し、学習曲線を比較

pass

---

## 9. Unit 0.0 総まとめ

### 学習したこと

| ノートブック | テーマ | キーポイント |
|-------------|--------|-------------|
| **70** | 微分の再発見 | 数値微分、偏微分、勾配ベクトル |
| **71** | 連鎖律の解剖 | 合成関数の微分、分岐と合流 |
| **72** | 計算グラフ | ノードとエッジ、トポロジカルソート |
| **73** | 逆伝播（スカラー） | backward()の実装、勾配チェック |
| **74** | 逆伝播（行列） | バッチ処理、行列微分、MLP |
| **75** | 学習ループ | SGD、Momentum、Adam、XOR問題 |
| **76** | 勾配の病理学 | 消失・爆発、初期化、鞍点 |

### 習得したスキル

1. **数学的理解**: 微分、連鎖律、勾配の意味
2. **実装力**: NumPyのみでのニューラルネットワーク構築
3. **デバッグ力**: 勾配チェック、モニタリング
4. **診断力**: 学習の問題を特定し対策する能力

### 次のステップへ

このユニットで学んだ「誤差逆伝播の物理」は、以下の発展的トピックの基礎となります：

- **CNN**: 畳み込み層の逆伝播
- **RNN/LSTM**: 時間方向の逆伝播（BPTT）
- **Transformer**: Attentionの勾配
- **自動微分ライブラリ**: PyTorch/JAXの内部理解

---

## 参考文献

1. Glorot, X., & Bengio, Y. (2010). Understanding the difficulty of training deep feedforward neural networks. *AISTATS*.
2. He, K., et al. (2015). Delving deep into rectifiers. *ICCV*.
3. Ioffe, S., & Szegedy, C. (2015). Batch normalization. *ICML*.
4. Kingma, D. P., & Ba, J. (2015). Adam: A method for stochastic optimization. *ICLR*.